# Load and Inspect the Datasets

In [2]:
# In a new notebook cell, run:
%pip install h3==3.7.7

  Using cached h3-3.7.7-cp311-cp311-macosx_11_0_arm64.whl.metadata (4.9 kB)
Using cached h3-3.7.7-cp311-cp311-macosx_11_0_arm64.whl (984 kB)
  Attempting uninstall: h3
    Found existing installation: h3 4.2.2
    Uninstalling h3-4.2.2:
      Successfully uninstalled h3-4.2.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import folium
from sklearn.neighbors import BallTree
from h3 import geo_to_h3, h3_to_geo_boundary   # type: ignore
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from folium.plugins import HeatMap

In [3]:

pd.set_option('display.max_columns', None)  # show all columns fully
pd.set_option('display.width', 0)            # no wrapping

In [4]:
sold_df = pd.read_csv('sold_listings_2.csv')
cleaned_df = pd.read_csv('cleaned_sale_data.csv')

print(sold_df.columns)
print(cleaned_df.columns)

/var/folders/_w/w1k0vkxd7tj4b3xwkrn8t00m0000gn/T/ipykernel_62635/1350226489.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  sold_df = pd.read_csv('sold_listings_2.csv')


Index(['id', 'source_property_id', 'source_listing_id', 'address', 'city',
       'state', 'postal_code', 'beds', 'baths', 'sqft', 'list_date',
       'sold_date', 'prev_sold_date', 'sold_price', 'status', 'lng', 'lat'],
      dtype='object')
Index(['source_property_id', 'address', 'city', 'state', 'postal_code', 'beds',
       'baths', 'sqft', 'lot_sqft', 'type', 'lat', 'lng', 'source_listing_id',
       'list_date', 'list_price', 'sold_date', 'sold_price', 'prev_sold_date',
       'prev_sold_price', 'dom', 'status', 'status_date', 'final_sale_date',
       'final_sale_price'],
      dtype='object')


In [5]:
cleaned_df = cleaned_df.drop(columns=['final_sale_date', 'final_sale_price'])
print(cleaned_df.columns)

Index(['source_property_id', 'address', 'city', 'state', 'postal_code', 'beds',
       'baths', 'sqft', 'lot_sqft', 'type', 'lat', 'lng', 'source_listing_id',
       'list_date', 'list_price', 'sold_date', 'sold_price', 'prev_sold_date',
       'prev_sold_price', 'dom', 'status', 'status_date'],
      dtype='object')


In [6]:
df = pd.concat([sold_df, cleaned_df], axis=0, ignore_index = True, sort=False)
print(df.shape)
print(df.columns)
display(df.head())

(696873, 23)
Index(['id', 'source_property_id', 'source_listing_id', 'address', 'city',
       'state', 'postal_code', 'beds', 'baths', 'sqft', 'list_date',
       'sold_date', 'prev_sold_date', 'sold_price', 'status', 'lng', 'lat',
       'lot_sqft', 'type', 'list_price', 'prev_sold_price', 'dom',
       'status_date'],
      dtype='object')


,id,source_property_id,source_listing_id,address,city,state,postal_code,beds,baths,sqft,list_date,sold_date,prev_sold_date,sold_price,status,lng,lat,lot_sqft,type,list_price,prev_sold_price,dom,status_date
0,581637.0,1491434519,2915769772,6737 Hellman Ave,Alta Loma,CA,91701.0,3,2.0,1579,2020-11-24,NaN,2020-06-30,NaN,sold,-117.602399,34.131953,NaN,NaN,NaN,NaN,NaN,NaN
1,333916.0,1478209481,2940270018,33201 Churchill St,Lake Elsinore,CA,92530.0,4,2.0,1537,2023-03-03,NaN,2022-03-15,NaN,sold,-117.343840,33.634889,NaN,NaN,NaN,NaN,NaN,NaN
2,333933.0,2866740577,2938719663,32077 Poppy Way,Lake Elsinore,CA,92532.0,3,2.0,1551,2023-03-03,NaN,2022-03-09,NaN,sold,-117.255584,33.661429,NaN,NaN,NaN,NaN,NaN,NaN
3,333940.0,2882443387,2939669667,29265 Crystal Ridge Ct,Lake Elsinore,CA,92530.0,3,2.0,2003,2023-03-03,NaN,2022-03-07,NaN,sold,-117.414660,33.694345,NaN,NaN,NaN,NaN,NaN,NaN
4,333943.0,9308919583,2938323322,29126 Red Alder,Lake Elsinore,CA,92530.0,3,2.5,1688,2023-03-03,NaN,2022-03-04,NaN,sold,-117.308430,33.642118,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#clean columns and remove duplicates
df = df.drop_duplicates()
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [8]:
# 1. Create unified fields
df['final_sale_price'] = df['sold_price'].fillna(df['prev_sold_price'])
df['final_sale_date']  = pd.to_datetime(df['sold_date'], errors='coerce') \
                    .fillna(pd.to_datetime(df['prev_sold_date'], errors='coerce'))

# 2. Run QA checks
mask_sold = df['sold_price'].notna()
print("Price-fill accuracy:",
      (df.loc[mask_sold, 'final_sale_price'] == df.loc[mask_sold, 'sold_price']).mean())

mask_date = df['sold_date'].notna()
print("Date-fill accuracy:",
      (df.loc[mask_date, 'final_sale_date'] == pd.to_datetime(df.loc[mask_date, 'sold_date'])).mean())

Price-fill accuracy: 1.0
Date-fill accuracy: 1.0


In [9]:
df.drop(columns=['sold_price', 'sold_date', 'prev_sold_price', 'prev_sold_date'], inplace=True)

In [10]:
df['list_date'] = pd.to_datetime(df['list_date'], errors='coerce')
df['days_on_market'] = (df['final_sale_date'] - df['list_date']).dt.days

df = df.drop(columns=['dom'], errors='ignore')
display(df.head(30))

,id,source_property_id,source_listing_id,address,city,state,postal_code,beds,baths,sqft,list_date,status,lng,lat,lot_sqft,type,list_price,status_date,final_sale_price,final_sale_date,days_on_market
0,581637.0,1491434519,2915769772,6737 Hellman Ave,Alta Loma,CA,91701.0,3,2.0,1579,2020-11-24,sold,-117.602399,34.131953,NaN,NaN,NaN,NaN,NaN,2020-06-30,-147.0
1,333916.0,1478209481,2940270018,33201 Churchill St,Lake Elsinore,CA,92530.0,4,2.0,1537,2023-03-03,sold,-117.343840,33.634889,NaN,NaN,NaN,NaN,NaN,2022-03-15,-353.0
2,333933.0,2866740577,2938719663,32077 Poppy Way,Lake Elsinore,CA,92532.0,3,2.0,1551,2023-03-03,sold,-117.255584,33.661429,NaN,NaN,NaN,NaN,NaN,2022-03-09,-359.0
3,333940.0,2882443387,2939669667,29265 Crystal Ridge Ct,Lake Elsinore,CA,92530.0,3,2.0,2003,2023-03-03,sold,-117.414660,33.694345,NaN,NaN,NaN,NaN,NaN,2022-03-07,-361.0
4,333943.0,9308919583,2938323322,29126 Red Alder,Lake Elsinore,CA,92530.0,3,2.5,1688,2023-03-03,sold,-117.308430,33.642118,NaN,NaN,NaN,NaN,NaN,2022-03-04,-364.0
5,333949.0,2527667060,2939589889,11 Ponte Negra,Lake Elsinore,CA,92532.0,4,3.5,3262,2023-06-06,sold,-117.291220,33.685211,NaN,NaN,NaN,NaN,NaN,2022-03-04,-459.0
6,333956.0,2311800280,2934704332,29254 Woodbridge St,Lake Elsinore,CA,92530.0,4,3.0,2582,2023-06-06,sold,-117.408063,33.694380,NaN,NaN,NaN,NaN,NaN,2022-03-02,-461.0
7,333963.0,2835624095,2934592157,31925 Hyacinth Ct,Lake Elsinore,CA,92532.0,4,3.0,2880,2023-03-03,sold,-117.260787,33.658497,NaN,NaN,NaN,NaN,NaN,2022-03-01,-367.0
8,333966.0,2491295500,2939431559,18 Ponte Sonata,Lake Elsinore,CA,92532.0,3,2.5,2364,2023-03-03,sold,-117.286284,33.686851,NaN,NaN,NaN,NaN,NaN,2022-02-28,-368.0
9,328381.0,2446658757,2922863737,611 LA Vereda Dr,La Habra,CA,90631.0,3,2.0,1612,2022-12-07,sold,-117.952580,33.944864,NaN,NaN,NaN,NaN,NaN,2020-12-09,-728.0


In [11]:
df = df.dropna(subset = ['final_sale_price', 'final_sale_date'])

df = df.drop_duplicates(subset = 'source_listing_id')

df = df.dropna(subset=['postal_code'])

In [12]:
# Impute or fill ancillary features
df['lot_sqft']   = df['lot_sqft'].fillna(df['lot_sqft'].median())
df['type']       = df['type'].fillna('unknown')

# Enforce proper dtypes
df['postal_code'] = df['postal_code'].astype(int)
df['beds']        = df['beds'].astype(int)
df['baths']       = df['baths'].astype(float)

# Quick missing-value report
print("Rows x Columns:", df.shape)
print(df.isnull().sum().sort_values(ascending=False).head(10))

Rows x Columns: (350368, 21)
id                  205205
status_date         145163
list_price          145163
address                 33
city                     3
lng                      0
final_sale_date          0
final_sale_price         0
type                     0
lot_sqft                 0
dtype: int64


In [13]:
display(df.head(30))

,id,source_property_id,source_listing_id,address,city,state,postal_code,beds,baths,sqft,list_date,status,lng,lat,lot_sqft,type,list_price,status_date,final_sale_price,final_sale_date,days_on_market
204601,633260.0,2265116786,2978949477,1054 Sykes Dr,San Jacinto,CA,92582,4,2.0,1872,2025-03-04,sold,-116.979189,33.788484,7260.0,unknown,NaN,NaN,475000.0,2025-04-13,40.0
204602,633147.0,1029172544,2979931074,29734 Windwood Cir,Temecula,CA,92591,3,2.0,1429,2025-03-29,sold,-117.145856,33.522267,7260.0,unknown,NaN,NaN,627000.0,2025-04-12,14.0
204603,632341.0,2660233480,2978182687,18120 Ford Ave,Desert Hot Springs,CA,92241,2,2.5,1690,2025-02-11,sold,-116.404601,33.917431,7260.0,unknown,NaN,NaN,565000.0,2025-04-12,60.0
204604,633513.0,1971139554,2978886548,408 San Marino Oaks,San Gabriel,CA,91775,4,3.5,3348,2025-03-02,sold,-118.104923,34.111212,7260.0,unknown,NaN,NaN,2100000.0,2025-04-12,41.0
204605,633399.0,9927165677,2980520141,124 Obsidian Loop S,Palm Springs,CA,92264,3,3.5,2466,2025-04-12,sold,-116.546050,33.809766,7260.0,unknown,NaN,NaN,1710000.0,2025-04-12,0.0
204606,633296.0,2677712676,2978565073,16622 Lucia Ln,Huntington Beach,CA,92647,4,2.0,1480,2025-02-21,sold,-117.978729,33.721102,7260.0,unknown,NaN,NaN,1170000.0,2025-04-11,49.0
204607,633279.0,1020436202,2976792081,7152 Sunlight Dr,Huntington Beach,CA,92647,5,4.0,2760,2025-01-02,sold,-118.003764,33.723455,7260.0,unknown,NaN,NaN,1650000.0,2025-04-11,99.0
204608,632670.0,2035139362,2978306703,6210 8th Ave,Los Angeles,CA,90043,2,1.0,993,2025-02-14,sold,-118.326166,33.983252,7260.0,unknown,NaN,NaN,650000.0,2025-04-11,56.0
204609,632634.0,1709012504,2978060454,6622 Royer Ave,West Hills,CA,91307,4,2.0,1528,2025-02-07,sold,-118.628141,34.190563,7260.0,unknown,NaN,NaN,1075000.0,2025-04-11,63.0
204610,632598.0,1968032846,2977816422,3926 Dozier St,Los Angeles,CA,90063,2,1.0,615,2025-01-31,sold,-118.181428,34.041107,7260.0,unknown,NaN,NaN,500000.0,2025-04-11,70.0


In [14]:
# filter to the 2020-2025 window
df = df[df['final_sale_date'].between('2020-01-01', '2025-12-31')].reset_index(drop=True)
print(df.shape)
display(df.head(30))

(349860, 21)


,id,source_property_id,source_listing_id,address,city,state,postal_code,beds,baths,sqft,list_date,status,lng,lat,lot_sqft,type,list_price,status_date,final_sale_price,final_sale_date,days_on_market
0,633260.0,2265116786,2978949477,1054 Sykes Dr,San Jacinto,CA,92582,4,2.0,1872,2025-03-04,sold,-116.979189,33.788484,7260.0,unknown,NaN,NaN,475000.0,2025-04-13,40.0
1,633147.0,1029172544,2979931074,29734 Windwood Cir,Temecula,CA,92591,3,2.0,1429,2025-03-29,sold,-117.145856,33.522267,7260.0,unknown,NaN,NaN,627000.0,2025-04-12,14.0
2,632341.0,2660233480,2978182687,18120 Ford Ave,Desert Hot Springs,CA,92241,2,2.5,1690,2025-02-11,sold,-116.404601,33.917431,7260.0,unknown,NaN,NaN,565000.0,2025-04-12,60.0
3,633513.0,1971139554,2978886548,408 San Marino Oaks,San Gabriel,CA,91775,4,3.5,3348,2025-03-02,sold,-118.104923,34.111212,7260.0,unknown,NaN,NaN,2100000.0,2025-04-12,41.0
4,633399.0,9927165677,2980520141,124 Obsidian Loop S,Palm Springs,CA,92264,3,3.5,2466,2025-04-12,sold,-116.546050,33.809766,7260.0,unknown,NaN,NaN,1710000.0,2025-04-12,0.0
5,633296.0,2677712676,2978565073,16622 Lucia Ln,Huntington Beach,CA,92647,4,2.0,1480,2025-02-21,sold,-117.978729,33.721102,7260.0,unknown,NaN,NaN,1170000.0,2025-04-11,49.0
6,633279.0,1020436202,2976792081,7152 Sunlight Dr,Huntington Beach,CA,92647,5,4.0,2760,2025-01-02,sold,-118.003764,33.723455,7260.0,unknown,NaN,NaN,1650000.0,2025-04-11,99.0
7,632670.0,2035139362,2978306703,6210 8th Ave,Los Angeles,CA,90043,2,1.0,993,2025-02-14,sold,-118.326166,33.983252,7260.0,unknown,NaN,NaN,650000.0,2025-04-11,56.0
8,632634.0,1709012504,2978060454,6622 Royer Ave,West Hills,CA,91307,4,2.0,1528,2025-02-07,sold,-118.628141,34.190563,7260.0,unknown,NaN,NaN,1075000.0,2025-04-11,63.0
9,632598.0,1968032846,2977816422,3926 Dozier St,Los Angeles,CA,90063,2,1.0,615,2025-01-31,sold,-118.181428,34.041107,7260.0,unknown,NaN,NaN,500000.0,2025-04-11,70.0


In [15]:
#remove extreme low-end outliers
floor = df['final_sale_price'].quantile(0.01)
df = df[df['final_sale_price'] >= floor]
print("Floor:", floor)
print(df.shape)

Floor: 235000.0
(346528, 21)


In [16]:
#building balltree for 0.33 mi neighbor search
def build_neighbor_tree(df, lat_col='lat', lng_col='lng', radius_miles=0.33):
    coords = np.radians(df[[lat_col, lng_col]].to_numpy())
    tree = BallTree(coords, metric='haversine')
    radius_rad = radius_miles / 3958.8
    return tree, coords, radius_rad

In [17]:
#query neighbors
def find_neighbors(tree, coords, radius_rad):
    inds = tree.query_radius(coords, r=radius_rad)
    return{i: list(neigh) for i, neigh in enumerate(inds)}

In [18]:
#compute local median & 75th percentile
def calculate_local_stats(df, neighbor_dict):
    medians, pct75 = [], []
    for i, neigh in neighbor_dict.items():
        prices = df.iloc[neigh]['final_sale_price']
        medians.append(prices.median())
        pct75.append(prices.quantile(0.75))
    df['local_median'] = medians
    df['local_75pct'] = pct75
    return df

In [19]:
#split pockets by top 25% of radius-computed 75th percentile
def classify_by_radius_pct(df, top_pct=0.25):
    cutoff = df['local_75pct'].quantile(1 - top_pct)
    df['market_type'] = np.where(df['local_75pct'] >= cutoff,
                                'Established', 'Emerging')
    print("Market Counts:", df['market_type'].value_counts().to_dict())
    return df

In [20]:
def smooth_mass(df, neighbor_dict):
    sm = []
    for i, neigh in neighbor_dict.items():
        mode = df['market_type'].iloc[neigh].mode()
        sm.append(mode.iloc[0] if not mode.empty else df['market_type'].iloc[i])
    df['market_type'] = sm
    return df

In [21]:
#flag any radius with an IQR outlier sale
def flag_growth_area(df, neighbor_dict, iqr_mul=1.5):
    flags = []
    for i, neigh in neighbor_dict.items():
        ps = df.iloc[neigh]['final_sale_price']
        q1, q3 = ps.quantile([0.25, 0.75])
        thr = q3 + iqr_mul * (q3 - q1)
        flags.append((ps > thr).any())
    df['growth_area'] = flags
    print("Growth areas flagged:", df['growth_area'].sum())
    return df

In [22]:
#identify which sale triggered each growth area
def find_trigger_sales(df, neighbor_dict, iqr_mul=1.5):
    triggers = []
    for i, neigh in neighbor_dict.items():
        ps = df.iloc[neigh]['final_sale_price']
        q1, q3 = ps.quantile([0.25, 0.75])
        thr = q3 + iqr_mul * (q3 - q1)
        out = ps[ps > thr]
        triggers.append((out - thr).idxmax() if not out.empty else None)

    df['trigger_idx'] = triggers
    df['trigger_address'] = df['trigger_idx'].apply(
        lambda i: df.at[i, 'address'] if pd.notnull(i) else 'N/A'
    )
    df['trigger_price']   = df['trigger_idx'].apply(
        lambda i: df.at[i, 'final_sale_price'] if pd.notnull(i) else None
    )
    df['trigger_date']    = df['trigger_idx'].apply(
        lambda i: df.at[i, 'final_sale_date'] if pd.notnull(i) else 'N/A'
    )
    df['is_trigger_sale'] = df.index.to_series() == df['trigger_idx']
    return df

In [23]:
tree, coords, rad = build_neighbor_tree(df)
nbrs = find_neighbors(tree, coords, rad)

df = calculate_local_stats(df, nbrs)
df = classify_by_radius_pct(df, top_pct=0.25)
df = smooth_mass(df, nbrs)
df = flag_growth_area(df, nbrs)
df = find_trigger_sales(df, nbrs)

Market Counts: {'Emerging': 259518, 'Established': 87010}
Growth areas flagged: 199615


In [24]:
#aggregate into h3 hexagon
H3_RES = 9
df['hex_id'] = df.apply(
    lambda r: geo_to_h3(r.lat, r.lng, H3_RES),
    axis =1
)

hex_stats = df.groupby('hex_id').agg(
    local_median = ('local_median', 'first'),
    local_75pct = ('local_75pct', 'first'),
    market_type = ('market_type', 'first'),
    growth_area = ('growth_area', 'first'),
    trigger_idx = ('trigger_idx', 'first')
).reset_index()

hex_stats['trigger_address'] = hex_stats['trigger_idx'].map(df['address'])
hex_stats['trigger_price'] = hex_stats['trigger_idx'].map(df['final_sale_price'])
hex_stats['trigger_date'] = hex_stats['trigger_idx'].map(df['final_sale_date'])

# Make sure all fields are JSON‐serializable
hex_stats['trigger_date']  = hex_stats['trigger_date'].apply(
    lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) and hasattr(x, 'strftime') else str(x)
)
hex_stats['trigger_price'] = hex_stats['trigger_price'].fillna('').astype(str)
hex_stats['local_median']  = hex_stats['local_median'].astype(float)
hex_stats['local_75pct']   = hex_stats['local_75pct'].astype(float)

# build HTML list of underlying properties
def make_prop_list(sub):
    items = []
    for _, r in sub.iterrows():
        items.append(
            f"<li>"
            f"<b>{r['address']}</b>, {r['city']} — "
            f"${r['final_sale_price']:,.0f} — "
            f"{int(r['beds'])}bd/{int(r['baths'])}ba — "
            f"{int(r.get('days_on_market',0))}d"
            f"</li>"
        )
    return "<ul style='margin:0;padding-left:1em;'>" + "".join(items) + "</ul>"

prop_lists = (
    df.groupby('hex_id')
      .apply(make_prop_list)
      .reset_index(name='properties_list')
)
hex_stats = hex_stats.merge(prop_lists, on='hex_id', how='left')


/var/folders/_w/w1k0vkxd7tj4b3xwkrn8t00m0000gn/T/ipykernel_62635/3877689003.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(make_prop_list)


In [ ]:
# m = folium.Map(
#     location=[df['lat'].mean(), df['lng'].mean()],
#     zoom_start=11,
#     tiles='cartodbpositron'
# )

# def style_hex(feature):
#     p      = feature['properties']
#     fill   = '#357edd' if p['market_type']=='Established' else '#31a354'
#     border = '#ff7800' if p['growth_area'] else '#444444'
#     return {
#         'fillColor':   fill,
#         'color':       border,
#         'weight':      0.8 if p['growth_area'] else 0.4,
#         'fillOpacity': 0.20
#     }

# features = []
# for _, row in hex_stats.iterrows():
#     boundary = h3_to_geo_boundary(row.hex_id, geo_json=True)
#     features.append({
#         'type': 'Feature',
#         'geometry': {'type':'Polygon','coordinates':[boundary]},
#         'properties': row.to_dict()
#     })

# folium.GeoJson(
#     {'type':'FeatureCollection','features':features},
#     style_function=style_hex,
#     tooltip=folium.GeoJsonTooltip(
#         fields=['market_type','local_median','local_75pct','growth_area'],
#         aliases=['Market Type:','Median Price:','75th-pct Price:','Growth Area:'],
#         localize=True
#     ),
#     popup=folium.GeoJsonPopup(
#         fields=['properties_list','trigger_address','trigger_price','trigger_date'],
#         aliases=['Properties:','Highlight Sale Addr:','Highlight Sale Price:','Highlight Sale Date:'],
#         labels=False, localize=True,
#         style="max-height:200px;overflow:auto;"
#     )
# ).add_to(m)

# # draw median-price labels on top
# for _, row in hex_stats.iterrows():
#     coords = h3_to_geo_boundary(row.hex_id, geo_json=True)
#     lats, lons = zip(*coords)
#     centroid = (sum(lats)/len(lats), sum(lons)/len(lons))
#     folium.map.Marker(
#         location=centroid,
#         icon=DivIcon(
#             icon_size=(150,36),
#             icon_anchor=(0,0),
#             html=f"""
#               <div style="
#                 font-size:12px;
#                 font-weight:bold;
#                 color:#000;
#                 background:rgba(255,255,255,0.8);
#                 padding:2px 4px;
#                 border-radius:3px;
#                 border:1px solid #555;
#                 pointer-events:none;
#               ">
#                 ${row['local_median']:,.0f}
#               </div>
#             """
#         )
#     ).add_to(m)

# # add red‐star markers for each trigger‐sale property
# growth_cluster = MarkerCluster(name='Growth‐Signal Sales').add_to(m)
# for _, r in df[df['is_trigger_sale']].iterrows():
#     folium.Marker(
#         location=(r['lat'], r['lng']),
#         icon=folium.Icon(color='red', icon='star', prefix='fa'),
#         popup=(
#             f"<b>{r['address']}</b><br>"
#             f"Sale: ${r['final_sale_price']:,.0f}<br>"
#             f"Pct above median: {(r['final_sale_price']/r['local_median']-1):.1%}"
#         )
#     ).add_to(growth_cluster)

# folium.LayerControl(collapsed=False).add_to(m)

# m.save('hex_market_map_with_stars.html')
# print("Saved hex_market_map_with_stars.html — open it in your browser")

Saved hex_market_map_with_stars.html — open it in your browser


In [25]:
# 4) RENDER MAP: HEX GRID (striped for growth), HEATMAP, LABELS & STARS
m = folium.Map(
    location=[df.lat.mean(), df.lng.mean()],
    zoom_start=11,
    tiles='cartodbpositron'
)

# Inject the SVG stripe pattern
stripe_svg = """
<svg width='0' height='0' style='position:absolute'>
  <defs>
    <pattern id='stripePattern' patternUnits='userSpaceOnUse'
             width='8' height='8' patternTransform='rotate(45)'>
      <line x1='0' y='0' x2='0' y2='6' stroke='#006400' stroke-width='2'/>
    </pattern>
  </defs>
</svg>
"""
m.get_root().header.add_child(folium.Element(stripe_svg))

# Heatmap layer (off by default)
# heat_layer = folium.FeatureGroup(name='Deal Density Heatmap', show=False)
# HeatMap(
#     df[['lat','lng']].values.tolist(),
#     radius=15,
#     blur=10,
#     max_zoom=12
# ).add_to(heat_layer)
# heat_layer.add_to(m)

# Style function: stripe fill for growth areas, solid otherwise
def style_hex_base(feature):
    p    = feature['properties']
    fill = '#357edd' if p['market_type']=='Established' else '#31a354'
    return {
        'fillColor':   fill,
        'color':       fill,
        'weight':      0,
        'fillOpacity': 0.35
    }

# style for stripe overlay: apply only on growth_area hexes
def style_hex_stripe(feature):
    return {
        'fillColor':   'url(#stripePattern)',
        'fillOpacity': 0.35,
        'color':       None,
        'weight':      0
    }
# 4a) Build GeoJSON features list
features = [{
    'type':'Feature',
    'geometry':{'type':'Polygon','coordinates':[
        h3_to_geo_boundary(row.hex_id, geo_json=True)
    ]},
    'properties': row.to_dict()
} for _, row in hex_stats.iterrows()]


# 4b) Add the hex layer once, outside the loop
base_layer = folium.FeatureGroup(name='Neighborhood Hexes', show=True)
folium.GeoJson(
    {'type':'FeatureCollection','features':features},
    style_function=style_hex_base,
    tooltip=folium.GeoJsonTooltip(
        fields=['market_type','local_median','local_75pct','growth_area'],
        aliases=['Type:','Median:','75th-pct:','Growth area:'],
        localize=True
    ),
    popup=folium.GeoJsonPopup(
        fields=['properties_list','trigger_address','trigger_price','trigger_date'],
        aliases=['Properties:','Highlight Addr:','Highlight Price:','Highlight Date:'],
        labels=False, localize=True,
        style="max-height:200px;overflow:auto;"
    )
).add_to(base_layer)
base_layer.add_to(m)

growth_feats = [f for f in features if f['properties']['growth_area']]
stripe_layer = folium.FeatureGroup(name='Growth Stripes', show=True)
folium.GeoJson(
    {'type':'FeatureCollection','features':growth_feats},
    style_function=style_hex_stripe,
    interactive=False    # ← crucial so clicks go to base_layer
).add_to(stripe_layer)
stripe_layer.add_to(m)

# 4c) Median price labels on top
for _, row in hex_stats.iterrows():
    boundary = h3_to_geo_boundary(row.hex_id, geo_json=True)
    lats, lons = zip(*boundary)
    centroid = (sum(lats)/len(lats), sum(lons)/len(lons))
    folium.map.Marker(
        location=centroid,
        icon=DivIcon(
            icon_size=(0,0),
            html=(
              f"<div style='font-size:11px;font-weight:bold;"
              f"background:rgba(255,255,255,0.8);padding:1px 3px;"
              f"border-radius:2px;pointer-events:none;'>"
              f"${row['local_median']:,.0f}</div>"
            )
        )
    ).add_to(m)

# 4d) Growth-signal star markers
growth_cluster = MarkerCluster(name='Growth‐Signal Sales', show=True).add_to(m)

# 2) Loop through each trigger sale and add a star marker
for _, r in df[df['is_trigger_sale']].iterrows():
    folium.Marker(
        location=(r['lat'], r['lng']),
        icon=folium.Icon(color='red', icon='star', prefix='fa'),
        tooltip=f"{r['address']} — ${r['final_sale_price']:,.0f}",
        popup=(
            f"<b>{r['address']}</b><br>"
            f"Sale: ${r['final_sale_price']:,.0f}<br>"
            f"Pct above median: {(r['final_sale_price']/r['local_median']-1):.1%}"
        )
    ).add_to(growth_cluster)

# 4e) Layer control & save
folium.LayerControl(collapsed=False).add_to(m)
m.save('hex_map_with_heat_and_noborders.html')
print("Saved hex_map_with_heat_and_noborders.html — open in your browser")

Saved hex_map_with_heat_and_noborders.html — open in your browser
